In [11]:
from bs4 import BeautifulSoup
import urllib.request
import numpy as np
import pandas as pd
import helper_functions as hf

## Add some team spellings to MTeamSpellings.csv manually

This is needed for joining our scraped data (sports-reference) with our provided data (kaggle). Think the best/quickest way to do this is just manually right now.

"Spellings" I'm manually adding to the MTeamSpellings.csv (which didn't have matches with the sports-reference dataset):
- california-baptist (1465)
- dixie-state (1469)
- north-alabama (1466)
- st-thomas-mn (1472)
- tarleton-state (1460)
- california-san-diego (1471)

In [13]:
unmatched_spellings_lst = [('california-baptist', 1465), ('dixie-state', 1469), ('north-alabama', 1466),
                           ('st-thomas-mn', 1472), ('tarleton-state', 1460), ('california-san-diego', 1471)]
hf.add_team_spellings(unmatched_spellings_lst)

AttributeError: module 'helper_functions' has no attribute 'add_team_spellings'

In [20]:
team_spellings = pd.read_csv('data/kaggle_data/MTeamSpellings.csv')

In [21]:
unmatched_spellings_lst = [('california-baptist', 1465), ('dixie-state', 1469), ('north-alabama', 1466),
                           ('st-thomas-mn', 1472), ('tarleton-state', 1460), ('california-san-diego', 1471)]
unmatched_spellings = pd.DataFrame({
    'TeamNameSpelling': [spelling for spelling, _ in unmatched_spellings_lst],
    'TeamID': [teamid for _, teamid in unmatched_spellings_lst]
})
unmatched_spellings

,TeamNameSpelling,TeamID
0,california-baptist,1465
1,dixie-state,1469
2,north-alabama,1466
3,st-thomas-mn,1472
4,tarleton-state,1460
5,california-san-diego,1471


In [22]:
full_spellings = pd.concat([team_spellings, unmatched_spellings], ignore_index=True)
full_spellings.drop_duplicates(inplace=True)
full_spellings.to_csv('data/kaggle_data/MTeamSpellings.csv', index=False)

In [23]:
team_spellings = pd.read_csv('data/kaggle_data/MTeamSpellings.csv')
team_spellings

,TeamNameSpelling,TeamID
0,a&m-corpus chris,1394
1,a&m-corpus christi,1394
2,abilene chr,1101
3,abilene christian,1101
4,abilene-christian,1101
...,...,...
1158,dixie-state,1469
1159,north-alabama,1466
1160,st-thomas-mn,1472
1161,tarleton-state,1460


## Scrape coaching data from sports-reference

In [16]:
def get_coach_df(yr):
    link = 'https://www.sports-reference.com/cbb/seasons/' + str(yr) + '-coaches.html'
    with urllib.request.urlopen(link) as url:
        page = url.read()
    soup = BeautifulSoup(page, "html.parser")
    coach_table = soup.find('table', {'id': 'coaches'})
    coach_tbody = coach_table.find('tbody')
    coach_rows = coach_tbody.find_all('tr')
    cols = ['Season','school','is_ap_pre_top_5', 'is_ap_pre_top_15', 'is_ap_pre_top_25','coach_WL_car','tourneys_car','sw16_car','ff_car','champ_car']
    coach_df = pd.DataFrame(columns=cols)
    for coach_row in coach_rows:
        tds = coach_row.find_all('td')
        if len(tds) > 10:
            school = tds[0].find('a')['href'].split('/')[3]
            is_ap_pre_top_5 = False if tds[6].text == '' else int(tds[6].text) <= 5
            is_ap_pre_top_15 = False if tds[6].text == '' else 5 < int(tds[6].text) <= 15
            is_ap_pre_top_25 = False if tds[6].text == '' else 15 < int(tds[6].text) <= 25
            # arbitrarily choosing 0.300 winning % if the coach for some reason has no winning % listed (probably a bad sign)
            coach_WL_car = 0.3 if tds[21].text == '' else float(tds[21].text)
            tourneys_car = 0 if tds[22].text == '' else int(tds[22].text)
            sw16_car = 0 if tds[23].text == '' else int(tds[23].text)
            ff_car = 0 if tds[24].text == '' else int(tds[24].text)
            champ_car = 0 if tds[25].text == '' else int(tds[25].text)
            new_row = pd.Series([yr, school, is_ap_pre_top_5, is_ap_pre_top_15, is_ap_pre_top_25, coach_WL_car, tourneys_car, sw16_car, ff_car, champ_car], index=cols)
            coach_df = coach_df.append(new_row, ignore_index=True)
    return(coach_df)

In [17]:
def sports_ref_join_to_kaggle(coach_df):
    team_spellings = pd.read_csv('data/kaggle_data/MTeamSpellings.csv')
    coach_joined = team_spellings.merge(coach_df, left_on='TeamNameSpelling', right_on='school')
    coach_joined.drop('TeamNameSpelling', axis=1, inplace=True)
    return(coach_joined)

In [28]:
def check_for_missing_spellings(coach_df, coach_joined):
    coach_comp = coach_df.merge(coach_joined, on='school', how='left')
    return(coach_comp[coach_comp['TeamNameSpelling'].isna()])

## Compiling dataset

In [19]:
coach_df = get_coach_df(2003)
coach_joined = sports_ref_join_to_kaggle(coach_df)
for yr in range(2004,2023):
    print(yr)
    new_coach_df = get_coach_df(yr)
    new_coach_joined = sports_ref_join_to_kaggle(new_coach_df)
    coach_joined = pd.concat([coach_joined, new_coach_joined], ignore_index=True)

2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


In [20]:
coach_joined.head()

,TeamID,Season,school,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car
0,1102,2003,air-force,False,False,False,0.341,0,0,0,0
1,1103,2003,akron,False,False,False,0.462,0,0,0,0
2,1104,2003,alabama,False,True,False,0.660,4,0,0,0
3,1105,2003,alabama-am,False,False,False,0.518,0,0,0,0
4,1412,2003,alabama-birmingham,False,False,False,0.618,0,0,0,0


In [21]:
coach_joined[coach_joined['Season'] == 2022]

,TeamID,Season,school,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car
6601,1101,2022,abilene-christian,False,False,False,0.697,0,0,0,0
6602,1102,2022,air-force,False,False,False,0.474,1,0,0,0
6603,1103,2022,akron,False,False,False,0.592,4,1,0,0
6604,1104,2022,alabama,False,True,False,0.668,5,1,0,0
6605,1105,2022,alabama-am,False,False,False,0.290,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6954,1460,2022,wright-state,False,False,False,0.587,5,0,0,0
6955,1461,2022,wyoming,False,False,False,0.633,1,0,0,0
6956,1462,2022,xavier,False,False,False,0.580,0,0,0,0
6957,1463,2022,yale,False,False,False,0.547,3,0,0,0


In [22]:
coach_joined.to_csv('data/generated_data/coach_features.csv', index=False)

## Some data checks

In [27]:
coach_joined[coach_joined['Season'] == 2022].iloc[140:170]

,TeamID,Season,school,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car
6741,1240,2022,jacksonville-state,False,False,False,0.599,4,0,0,0
6742,1241,2022,james-madison,False,False,False,0.574,0,0,0,0
6743,1242,2022,kansas,True,False,False,0.768,23,13,3,1
6744,1243,2022,kansas-state,False,False,False,0.622,13,4,1,0
6745,1244,2022,kennesaw-state,False,False,False,0.226,0,0,0,0
6746,1245,2022,kent-state,False,False,False,0.612,1,0,0,0
6747,1246,2022,kentucky,False,True,False,0.771,21,15,6,1
6748,1247,2022,la-salle,False,False,False,0.388,0,0,0,0
6749,1248,2022,lafayette,False,False,False,0.455,3,0,0,0
6750,1249,2022,lamar,False,False,False,0.335,0,0,0,0
